In [1]:
import pickle
import numpy as np
import pandas as pd
from IPython.display import  display,Image
import tensorflow as tf
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Input
from keras.models import Model

from keras.layers.merge import concatenate
from keras.utils import plot_model

/Users/heyifan/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


# Part 1 Read data

In [2]:
df = pd.read_csv("/Users/heyifan/jupyter notebook/df_featured.csv", index_col = "index")
df.head()

,Name,date,open,high,low,close,volume,return,vol_change
index,,,,,,,,,
2013-03,A,2013-03-01,41.18,41.9800,40.73,41.93,3089323,0.075125,-0.409335
2013-04,A,2013-04-01,41.83,41.9771,40.79,40.93,2541331,0.024432,0.215632
2013-05,A,2013-05-01,41.42,41.7400,41.26,41.31,2726213,-0.009199,-0.067816
2013-06,A,2013-06-03,45.65,45.8400,45.04,45.51,3677473,-0.092287,-0.258672
2013-07,A,2013-07-01,43.05,43.7700,42.91,43.59,4283821,0.044047,-0.141544


# Part 2 Data engineering
Take one stock as example

先根据return来提取feature 

根据过去11个月的return来预测下一个月的return

In [4]:
A = df[df["Name"] == "A"]
print(f"shape for stock A: {A.shape}")
A.head(20)

shape for stock A: (60, 9)


,Name,date,open,high,low,close,volume,return,vol_change
index,,,,,,,,,
2013-03,A,2013-03-01,41.18,41.9800,40.730,41.93,3089323,0.075125,-0.409335
2013-04,A,2013-04-01,41.83,41.9771,40.790,40.93,2541331,0.024432,0.215632
2013-05,A,2013-05-01,41.42,41.7400,41.260,41.31,2726213,-0.009199,-0.067816
2013-06,A,2013-06-03,45.65,45.8400,45.040,45.51,3677473,-0.092287,-0.258672
2013-07,A,2013-07-01,43.05,43.7700,42.910,43.59,4283821,0.044047,-0.141544
2013-08,A,2013-08-01,45.51,46.2000,45.320,46.04,2745041,-0.053215,0.560567
2013-09,A,2013-09-03,47.04,47.5100,46.615,46.93,1565632,-0.018964,0.753312
2013-10,A,2013-10-01,50.70,52.0800,50.700,51.90,2997047,-0.095761,-0.477608
2013-11,A,2013-11-01,50.92,51.5800,50.840,51.10,1931035,0.015656,0.552042


rolling window的概念：https://blog.csdn.net/maymay_/article/details/80241627

In [5]:
def cnn_process(df,rollSize=12):
    if df.shape[0] < rollSize: # 如果股票的数据不足12个月则剔除这只股票 
        return None
    else:
        train_x = []
        train_y = []
        for i in range(df.shape[0]-rollSize):
            x = df[['return','vol_change']][i:i+rollSize-1].values.tolist()  ### x就是以12个月为单位的rolling取值；x[1]：第一组12个月的值，x[2]：第二组12个月的值
            y = df['return'][i+rollSize]  ### y是return率取完一个rolling window后下一个值 

            train_x.append(x)
            train_y.append([y])
        return train_x,train_y

In [7]:
print("X data generated")
cnn_process(A)[0][:3] # a股票的前三组rolling取值 第一列为return；第二列为vol_change 

X data generated


[[[0.07512520868113515, -0.40933499022277703],
  [0.024431956999755755, 0.21563188738499628],
  [-0.009198741224885064, -0.06781641786610215],
  [-0.09228740936058, -0.25867219147496123],
  [0.044046799724707375, -0.14154372930148107],
  [-0.05321459600347511, 0.5605672192145763],
  [-0.018964415086298758, 0.7533117616400278],
  [-0.09576107899807318, -0.477608459260065],
  [0.015655577299412915, 0.5520417807030944],
  [-0.03965420033828226, 0.3233545573353016],
  [-0.05337128624799858, -0.23847851953907814]],
 [[0.024431956999755755, 0.21563188738499628],
  [-0.009198741224885064, -0.06781641786610215],
  [-0.09228740936058, -0.25867219147496123],
  [0.044046799724707375, -0.14154372930148107],
  [-0.05321459600347511, 0.5605672192145763],
  [-0.018964415086298758, 0.7533117616400278],
  [-0.09576107899807318, -0.477608459260065],
  [0.015655577299412915, 0.5520417807030944],
  [-0.03965420033828226, 0.3233545573353016],
  [-0.05337128624799858, -0.23847851953907814],
  [0.00106856634

In [8]:
# 因为rolling window为12，所以头12支股票是没有办法取rolling组的；一共有60支股票，所以一共只有60-12=48个rolling组 
len( cnn_process(A)[0])

48

In [9]:
print("Y data generated:")
cnn_process(A)[1][:3] # cnn_process[0]表示rolling window；cnn_process[1]表示所有y的取值；仅为return的值 

Y data generated:


[[-0.009874801622288931], [0.006210078069552871], [0.034697998898476134]]

### Groupby & apply to all stocks

In [10]:
df_X_Y = df.groupby('Name').apply(lambda x: cnn_process(x,rollSize=12))

In [11]:
df_X_Y.head()

Name
A       ([[[0.07512520868113515, -0.40933499022277703]...
AAL     ([[[0.08376193975018363, 0.13972183060405596],...
AAP     ([[[0.033128191698310834, 0.02004755528734337]...
AAPL    ([[[0.10339747331927283, 0.14698706280343174],...
ABBV    ([[[-0.04125892620999738, 0.5520446606645322],...
dtype: object

In [12]:
raw_x = []
raw_y = []
ID_2_stock = {}
for ID in range(len(df_X_Y.values)):
    if df_X_Y.values[ID]: ## not none
        stock = df_X_Y.index.values[ID]
        ID_2_stock.update({ID:stock})
        stockData = df_X_Y[stock]
        raw_x.append(stockData[0])
        raw_y.append(stockData[1])
x = np.vstack(raw_x)
y = np.vstack(raw_y)

In [13]:
ID_2_stock

{0: 'A',
 1: 'AAL',
 2: 'AAP',
 3: 'AAPL',
 4: 'ABBV',
 5: 'ABC',
 6: 'ABT',
 7: 'ACN',
 8: 'ADBE',
 9: 'ADI',
 10: 'ADM',
 11: 'ADP',
 12: 'ADS',
 13: 'ADSK',
 14: 'AEE',
 15: 'AEP',
 16: 'AES',
 17: 'AET',
 18: 'AFL',
 19: 'AGN',
 20: 'AIG',
 21: 'AIV',
 22: 'AIZ',
 23: 'AJG',
 24: 'AKAM',
 25: 'ALB',
 26: 'ALGN',
 27: 'ALK',
 28: 'ALL',
 29: 'ALLE',
 30: 'ALXN',
 31: 'AMAT',
 32: 'AMD',
 33: 'AME',
 34: 'AMG',
 35: 'AMGN',
 36: 'AMP',
 37: 'AMT',
 38: 'AMZN',
 39: 'ANDV',
 40: 'ANSS',
 41: 'ANTM',
 42: 'AON',
 43: 'AOS',
 44: 'APA',
 45: 'APC',
 46: 'APD',
 47: 'APH',
 49: 'ARE',
 50: 'ARNC',
 51: 'ATVI',
 52: 'AVB',
 53: 'AVGO',
 54: 'AVY',
 55: 'AWK',
 56: 'AXP',
 57: 'AYI',
 58: 'AZO',
 59: 'BA',
 60: 'BAC',
 61: 'BAX',
 62: 'BBT',
 63: 'BBY',
 64: 'BDX',
 65: 'BEN',
 66: 'BF.B',
 69: 'BIIB',
 70: 'BK',
 71: 'BLK',
 72: 'BLL',
 73: 'BMY',
 74: 'BRK.B',
 75: 'BSX',
 76: 'BWA',
 77: 'BXP',
 78: 'C',
 79: 'CA',
 80: 'CAG',
 81: 'CAH',
 82: 'CAT',
 83: 'CB',
 84: 'CBG',
 85: 'CBOE',


In [18]:
len(ID_2_stock)

500

In [14]:
x

array([[[ 0.07512521, -0.40933499],
        [ 0.02443196,  0.21563189],
        [-0.00919874, -0.06781642],
        ...,
        [ 0.01565558,  0.55204178],
        [-0.0396542 ,  0.32335456],
        [-0.05337129, -0.23847852]],

       [[ 0.02443196,  0.21563189],
        [-0.00919874, -0.06781642],
        [-0.09228741, -0.25867219],
        ...,
        [-0.0396542 ,  0.32335456],
        [-0.05337129, -0.23847852],
        [ 0.00106857, -0.34585348]],

       [[-0.00919874, -0.06781642],
        [-0.09228741, -0.25867219],
        [ 0.0440468 , -0.14154373],
        ...,
        [-0.05337129, -0.23847852],
        [ 0.00106857, -0.34585348],
        [-0.0098748 ,  0.17717561]],

       ...,

       [[-0.03885273,  1.05128844],
        [-0.08266468,  0.1876882 ],
        [-0.0312726 ,  0.4366541 ],
        ...,
        [ 0.00885383, -0.41742659],
        [-0.01192938, -0.09235378],
        [-0.01380392, -0.23442479]],

       [[-0.08266468,  0.1876882 ],
        [-0.0312726 ,  0.43

In [15]:
x.shape # 一共抽取出多于12个月历史记录的股票500支，每支股票都有48个rolling window

(23476, 11, 2)

In [16]:
y.shape

(23476, 1)

# Part 3: Modeling
## Part 3.1 Product feature extraction based on CNN model with 1 feature("return")
### Part 3.1.1 Define some default parameters

In [20]:
n_features = 2 ### number of features, here we are using return & volatility
n_outputs = 1 ### number of outputs, here we only care the next month(step) return
n_timesteps = 11 ### number of months look back, here we are using 12 - 1 = 11
verbose = 1 ### how model training progress bar will show # print的方式 
epochs = 100 
batch_size =  2000 ### batch training 

### Part 3.1.2 Build CNN models based on "return" features <br>
#### The output layer will be the prediction of  next month return based on last n months, so the last layer [latent1, latent2, latent3,...] will be the feature that predict the return, so we could use it as stock features

In [22]:
x[:,:,0].shape # 这样操作会少一个dimension 

(23476, 11)

In [21]:
x_1 = x[:,:,0]  ### x_1 will lose 1 dim, since shape of x is (23476, 11, 2), so we need to reshape it to be (23476, 11, 1) 这里我们只使用return作为feature进行预测
x_1 = np.expand_dims(x_1,2) # 通过该操作来保存矩阵维度的完整 
print(f"x_1 shape: {x_1.shape}")

x_1 shape: (23476, 11, 1)


In [23]:
epochs = 500
batch_size = 3000
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(n_timesteps,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu',name="featureLayer")) # 将模型变得平缓 
model.add(Dense(1)) # 最终只预测一个y，所以让模型只输出一个值 
model.compile(loss='mse', optimizer='adam') # 确定优化方式

# fit network
model.fit(x_1, y, epochs=epochs, validation_split=0.2,batch_size=batch_size, verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 18780 samples, validate on 4696 samples
Epoch 1/500
18780/18780 [==============================] - 0s 15us/step - loss: 0.0054 - val_loss: 0.0048
Epoch 2/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0053 - val_loss: 0.0048
Epoch 3/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0052 - val_loss: 0.0048
Epoch 4/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0052 - val_loss: 0.0047
Epoch 5/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0052 - val_loss: 0.0047
Epoch 6/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0052 - val_loss: 0.0047
Epoch 7/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0052 - val_loss: 0.0047
Epoch 8/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0052 - val_los

18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 73/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 74/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 75/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 76/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 77/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 78/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 79/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 80/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0050 - val_loss: 0.0046
Epoch 81/500
18780/18780 [==============================] -

Epoch 149/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 150/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 151/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 152/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 153/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 154/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 155/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 156/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 157/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 158/500
18780/18780 [==========

18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 226/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 227/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 228/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 229/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 230/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 231/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 232/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 233/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 234/500
18780/18780 [========================

18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 302/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 303/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 304/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 305/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 306/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 307/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 308/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 309/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 310/500
18780/18780 [========================

18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 378/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 379/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 380/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 381/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 382/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 383/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 384/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 385/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 386/500
18780/18780 [========================

18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 454/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 455/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 456/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 457/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 458/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 459/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 460/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 461/500
18780/18780 [==============================] - 0s 2us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 462/500
18780/18780 [========================

### Part 3.1.3 Calculate last layer values as features

In [24]:
### Build new feature model, define input and output layer
featureModel= Model(inputs=model.input,outputs=model.get_layer("featureLayer").output) # 提取feature 

In [25]:
### make prediction
featureMatrix = featureModel.predict(x_1)

In [26]:
featureMatrix.shape # 10位latened feature 

(23476, 10)

## Part 3.2 Product feature extraction based on CNN model with 2 features("return","\vol_change")
将模型变得复杂起来，传入return和volume change两个参数

In [27]:
in_layers, out_layers = list(), list()

for i in range(n_features):
    inputs = Input(shape=(n_timesteps,1))
    conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)
    conv2 = Conv1D(filters=32, kernel_size=3, activation='relu')(conv1)
    pool1 = MaxPooling1D(pool_size=2)(conv2)
    flat = Flatten()(pool1)
    # store layers
    in_layers.append(inputs)
    out_layers.append(flat)
# merge heads
merged = concatenate(out_layers)
# interpretation
dense1 = Dense(100, activation='relu')(merged)
dense2 = Dense(10, activation='relu',name="featureLayer")(dense1)

outputs = Dense(n_outputs)(dense2)
model = Model(inputs=in_layers, outputs=outputs)
# compile model
model.compile(loss='mse', optimizer='adam')
# plot the model
try:
    plot_model(model, show_shapes=True, to_file='imgs/multiheaded_cnn.png')
except:
    print("you will need to input <brew install graphviz> in the terminal to plot the graph")
# fit network

input_data = [x[:,:,i].reshape((x.shape[0],n_timesteps,1)) for i in range(n_features)]

model.fit(x=input_data,y=y,validation_split=0.3,epochs=epochs,batch_size=batch_size,verbose=verbose)

you will need to input <brew install graphviz> in the terminal to plot the graph
Train on 16433 samples, validate on 7043 samples
Epoch 1/500
16433/16433 [==============================] - 1s 44us/step - loss: 0.0088 - val_loss: 0.0049
Epoch 2/500
16433/16433 [==============================] - 0s 14us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 3/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0055 - val_loss: 0.0046
Epoch 4/500
16433/16433 [==============================] - 0s 15us/step - loss: 0.0054 - val_loss: 0.0046
Epoch 5/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 6/500
16433/16433 [==============================] - 0s 14us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 7/500
16433/16433 [==============================] - 0s 14us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 8/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 9/500
16433/1643

16433/16433 [==============================] - 0s 13us/step - loss: 0.0050 - val_loss: 0.0045
Epoch 77/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0050 - val_loss: 0.0045
Epoch 78/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0050 - val_loss: 0.0045
Epoch 79/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0050 - val_loss: 0.0045
Epoch 80/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0050 - val_loss: 0.0044
Epoch 81/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0049 - val_loss: 0.0044
Epoch 82/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0049 - val_loss: 0.0045
Epoch 83/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0049 - val_loss: 0.0044
Epoch 84/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0049 - val_loss: 0.0044
Epoch 85/500
16433/16433 [========================

16433/16433 [==============================] - 0s 13us/step - loss: 0.0039 - val_loss: 0.0047
Epoch 153/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0039 - val_loss: 0.0047
Epoch 154/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0039 - val_loss: 0.0048
Epoch 155/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0039 - val_loss: 0.0048
Epoch 156/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0039 - val_loss: 0.0048
Epoch 157/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0038 - val_loss: 0.0049
Epoch 158/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0039 - val_loss: 0.0048
Epoch 159/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0038 - val_loss: 0.0051
Epoch 160/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0039 - val_loss: 0.0048
Epoch 161/500
16433/16433 [===============

Epoch 228/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0032 - val_loss: 0.0054
Epoch 229/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0031 - val_loss: 0.0054
Epoch 230/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0031 - val_loss: 0.0053
Epoch 231/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0031 - val_loss: 0.0054
Epoch 232/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0031 - val_loss: 0.0053
Epoch 233/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0030 - val_loss: 0.0053
Epoch 234/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0030 - val_loss: 0.0054
Epoch 235/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0030 - val_loss: 0.0053
Epoch 236/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0030 - val_loss: 0.0053
Epoch 237/500
16433/16433 [=

Epoch 304/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0025 - val_loss: 0.0059
Epoch 305/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0025 - val_loss: 0.0058
Epoch 306/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0025 - val_loss: 0.0058
Epoch 307/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0025 - val_loss: 0.0059
Epoch 308/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0025 - val_loss: 0.0059
Epoch 309/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0025 - val_loss: 0.0060
Epoch 310/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0025 - val_loss: 0.0059
Epoch 311/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0025 - val_loss: 0.0061
Epoch 312/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0025 - val_loss: 0.0059
Epoch 313/500
16433/16433 [=

Epoch 380/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0022 - val_loss: 0.0063
Epoch 381/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0022 - val_loss: 0.0062
Epoch 382/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0022 - val_loss: 0.0064
Epoch 383/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0022 - val_loss: 0.0062
Epoch 384/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0022 - val_loss: 0.0065
Epoch 385/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0021 - val_loss: 0.0064
Epoch 386/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0021 - val_loss: 0.0064
Epoch 387/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0021 - val_loss: 0.0063
Epoch 388/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0021 - val_loss: 0.0063
Epoch 389/500
16433/16433 [=

Epoch 456/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0019 - val_loss: 0.0066
Epoch 457/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0019 - val_loss: 0.0066
Epoch 458/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0019 - val_loss: 0.0069
Epoch 459/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0019 - val_loss: 0.0069
Epoch 460/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0019 - val_loss: 0.0068
Epoch 461/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0019 - val_loss: 0.0067
Epoch 462/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0019 - val_loss: 0.0071
Epoch 463/500
16433/16433 [==============================] - 0s 13us/step - loss: 0.0019 - val_loss: 0.0072
Epoch 464/500
16433/16433 [==============================] - 0s 12us/step - loss: 0.0019 - val_loss: 0.0070
Epoch 465/500
16433/16433 [=

In [28]:
### Build new feature model, define input and output layer
featureModel2= Model(inputs=in_layers,outputs=model.get_layer("featureLayer").output)

In [29]:
featureMatrix2 = featureModel2.predict(input_data)

In [30]:
featureMatrix2.shape

(23476, 10)

# Part 4: Modulize 
## Part 4.1 Modulize process part

In [31]:
class processor:
    def cnn_process(self,df,rollSize=12):
        if df.shape[0] < rollSize:
            return None
        else:
            train_x = []
            train_y = []
            for i in range(df.shape[0]-rollSize):
                x = df[['return','vol_change']].values[i:i+rollSize-1].tolist()  ### generate x
                y = df['return'].values[i+rollSize]  ### generate y

                train_x.append(x)
                train_y.append([y])
            return train_x,train_y

    def generate_cnn_input(self,df):            
        df_X_Y = df.groupby('Name').apply(lambda x: self.cnn_process(x,rollSize=12))
        raw_x = []
        raw_y = []
        ID_2_stock = {}
        for ID in range(len(df_X_Y.values)):
            if df_X_Y.values[ID]: ## not none
                stock = df_X_Y.index.values[ID]
                ID_2_stock.update({ID:stock})
                stockData = df_X_Y[stock]
                raw_x.append(stockData[0])
                raw_y.append(stockData[1])
        y = np.vstack(raw_y)
        x = np.vstack(raw_x)
        return x,y,ID_2_stock

In [32]:
myProcessor = processor()
x,y,ID_2_stock = myProcessor.generate_cnn_input(df)

## Part 4.2 Modulize model part

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Input
from keras.models import Model
import numpy as np
import pandas as pd
from keras.layers.merge import concatenate
from keras.utils import plot_model

class extractor:
    def train(self,x,y,n_timesteps,n_features,verbose=1,batch_size=2000,epochs=500,n_outputs=1):
        self.x = x
        self.y = y
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.verbose = verbose
        self.batch_size = batch_size
        self.epochs = epochs
        self.n_outputs = n_outputs
        if self.n_timesteps <= 6: 
            raise Exception("Need at least 6 timesteps, Conv & Pool layer will reduce dimension by (3-1) + (3-1) + 2 = 6, check model structure below ")
        
        self.build_model()
        self.fit()
        self.build_feature_model()
        
    def build_model(self):
        in_layers, out_layers = list(), list()

        for i in range(self.n_features):
            inputs = Input(shape=(self.n_timesteps,1))
            conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)
            conv2 = Conv1D(filters=32, kernel_size=3, activation='relu')(conv1)
            pool1 = MaxPooling1D(pool_size=2)(conv2)
            flat = Flatten()(pool1)
            # store layers
            in_layers.append(inputs)
            out_layers.append(flat)
        # merge headss
        merged = concatenate(out_layers)
        # interpretation
        dense1 = Dense(100, activation='relu')(merged)
        dense2 = Dense(10, activation='relu',name="featureLayer")(dense1)

        outputs = Dense(self.n_outputs)(dense2)
        self.model = Model(inputs=in_layers, outputs=outputs)
        # compile model
        self.model.compile(loss='mse', optimizer='adam')
        # plot the model
        try:
            plot_model(self.model, show_shapes=True, to_file='imgs/multiheaded_cnn.png')
        except:
            print("you will need to input <brew install graphviz> in the terminal to plot the graph")
        # fit network

    def fit(self):
        self.input_data = [self.x[:,:,i].reshape((self.x.shape[0],self.n_timesteps,1)) for i in range(self.n_features)]

        self.model.fit(x=self.input_data,y=self.y,validation_split=0.3,epochs=self.epochs,batch_size=self.batch_size,verbose=self.verbose)
    
    def build_feature_model(self):
        self.featureModel = Model(inputs=self.model.input,outputs=self.model.get_layer("featureLayer").output)
        
    def get_feature_matrix(self,x):
        input_data = [x[:,:,i].reshape((x.shape[0],self.n_timesteps,1)) for i in range(self.n_features)]

        featureMatrix = self.featureModel.predict(input_data)
        return featureMatrix

In [34]:
myExtractor = extractor()
myExtractor.train(x,y,n_timesteps=11,n_features=2,verbose=1,batch_size=2000,epochs=10)

you will need to input <brew install graphviz> in the terminal to plot the graph
Train on 16433 samples, validate on 7043 samples
Epoch 1/10
16433/16433 [==============================] - 1s 48us/step - loss: 0.0064 - val_loss: 0.0047
Epoch 2/10
16433/16433 [==============================] - 0s 13us/step - loss: 0.0054 - val_loss: 0.0046
Epoch 3/10
16433/16433 [==============================] - 0s 13us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 4/10
16433/16433 [==============================] - 0s 13us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 5/10
16433/16433 [==============================] - 0s 13us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 6/10
16433/16433 [==============================] - 0s 13us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 7/10
16433/16433 [==============================] - 0s 13us/step - loss: 0.0053 - val_loss: 0.0045
Epoch 8/10
16433/16433 [==============================] - 0s 15us/step - loss: 0.0052 - val_loss: 0.0045
Epoch 9/10
16433/16433 [======

In [35]:
# 给用户推荐产品一般都是用的产品最新的数据，所以从数据集中抽取最近11个月的return rate和vol_change
recent_xs = df.groupby("Name").apply(lambda x: x[['return','vol_change']][-11:].values) 
recent_xs.head()

Name
A       [[-0.021804511278195604, 0.30729241588489953],...
AAL     [[0.10930506478209656, 0.4361392651433305], [0...
AAP     [[0.07683848797250857, -0.4887237492222128], [...
AAPL    [[-0.027209464161447428, 0.8220307265479732], ...
ABBV    [[-0.033830539750884236, 0.4795471359833072], ...
dtype: object

In [36]:
recent_x = []
for stock in ID_2_stock.values():
    recent_x.append(recent_xs[stock])
recent_x = np.array(recent_x)

In [37]:
recent_x[0].shape

(11, 2)

In [38]:
recent_x.shape 

(500, 11, 2)

In [39]:
featureMatrix = myExtractor.get_feature_matrix(recent_x)

In [40]:
featureMatrix.shape

(500, 10)

In [42]:
# feature matrix 
# 500支股票最近11个月的latend feature 
pd.DataFrame(featureMatrix)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.042652,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.117835,0.0,0.0,0.000000,0.0,0.019460,0.0,0.0,0.0
2,0.0,0.018618,0.0,0.0,0.033598,0.0,0.079799,0.0,0.0,0.0
3,0.0,0.082732,0.0,0.0,0.000000,0.0,0.040619,0.0,0.0,0.0
4,0.0,0.059072,0.0,0.0,0.000000,0.0,0.003174,0.0,0.0,0.0
5,0.0,0.088238,0.0,0.0,0.000000,0.0,0.008651,0.0,0.0,0.0
6,0.0,0.099311,0.0,0.0,0.000000,0.0,0.061552,0.0,0.0,0.0
7,0.0,0.091309,0.0,0.0,0.000000,0.0,0.007823,0.0,0.0,0.0
8,0.0,0.035357,0.0,0.0,0.000000,0.0,0.014634,0.0,0.0,0.0
9,0.0,0.220275,0.0,0.0,0.000000,0.0,0.118809,0.0,0.0,0.0


In [46]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [48]:
save_obj(featureMatrix,"/Users/heyifan/jupyter notebook/step5_featureMatrix")